In [1]:
from rlnoise.dataset import Dataset, dataset_folder, figures_folder
import numpy as np
import matplotlib.pyplot as plt

Create and save dataset (1st use)

In [5]:
nqubits = 1
ngates = 5
ncirc = 100

# create dataset
print("Creating random cicuits")
dataset = Dataset(
    n_circuits=ncirc,
    n_gates=ngates,
    n_qubits=nqubits,
)
print("Generating cicuits representation")
circuits_repr=dataset.generate_dataset_representation()
print("Adding noise")
dataset.add_noise(noise_params=0.07)
print("Measuring observables")
labels=dataset.generate_labels()

np.save(dataset_folder() + '/dataset_1q.npy', circuits_repr)
np.save(dataset_folder() + '/labels_1q.npy', labels)
print("Circuits and labels saved in: ", dataset_folder())

[Qibo 0.1.10.dev0|INFO|2023-02-08 16:07:21]: Using tensorflow backend on /device:CPU:0
2023-02-08 16:07:21.064911: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


Creating random cicuits
Generating cicuits representation
Adding noise
Measuring observables


2023-02-08 16:07:21.065045: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-08 16:07:21.065068: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-02-08 16:07:21.065086: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-02-08 16:07:21.065105: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-02-08 16:07:21.065122: W tensorflow/stream_executor/platform/default/dso_loader.cc:64

Circuits and labels saved in:  /home/simonebordoni/QIBO/rl-noisemodel/dataset


Load existing dataset

In [2]:
circuits_repr = np.load(dataset_folder() + '/dataset_1q.npy')
labels = np.load(dataset_folder() + '/labels_1q.npy')

print("Dataset loaded")
print("Number of circuits: ", len(circuits_repr))

Dataset loaded
Number of circuits:  100


Visualize circuit representation and observables distributions moments

In [7]:
element=0

print("Circuit representation:")
print(circuits_repr[element])
print("Observables distribution moments:")
print(labels[element])

Circuit representation:
[[0.         0.54156335]
 [0.         0.37195697]
 [0.         0.21414577]
 [1.         0.80536387]
 [0.         0.53812583]]
Observables distribution moments:
[[0.328      0.006464  ]
 [0.2062     0.00944556]
 [0.199      0.009235  ]]


Create RL environment

In [3]:
from rlnoise.envs.gym_env import CircuitsGym

env=CircuitsGym(circuits_repr, labels)

Define ANN

In [4]:
import tensorflow as tf
from tensorflow import keras

num_inputs = env.observation_space.shape
num_actions = env.action_space.n

# CNN model
inputs = keras.layers.Input(shape=num_inputs)
common = keras.layers.Conv1D(8, (1,), activation="relu")(inputs)
common = keras.layers.Conv1D(4, (3,), padding='same', activation="relu")(common)
common = keras.layers.Flatten()(common)
action = keras.layers.Dense(num_actions, activation="softmax", name='Actor')(common)
critic = keras.layers.Dense(1, name='Critic')(common)

model = keras.Model(inputs=inputs, outputs=[action, critic])
model.summary()

2023-02-08 16:37:20.151123: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/simonebordoni/anaconda3/envs/qibo/lib/python3.10/site-packages/cv2/../../lib64:


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 5, 4)]       0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 5, 8)         40          ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 5, 4)         100         ['conv1d[0][0]']                 
                                                                                                  
 flatten (Flatten)              (None, 20)           0           ['conv1d_1[0][0]']               
                                                                                              

2023-02-08 16:37:21.056912: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/simonebordoni/anaconda3/envs/qibo/lib/python3.10/site-packages/cv2/../../lib64:
2023-02-08 16:37:21.056950: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/simonebordoni/anaconda3/envs/qibo/lib/python3.10/site-packages/cv2/../../lib64:
2023-02-08 16:37:21.056972: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/simonebordoni/anaconda3/envs/qibo/lib/python3.10/site-packages/cv2/../../lib64:
20

Define Agent

In [5]:
from rlnoise.agents.actor_critic import AC_agent

agent=AC_agent(model, env)

Train Agent

In [6]:
optimizer = keras.optimizers.Adam(learning_rate=0.01)
episodes=100

reward_history, final_observation_history = agent.train(episodes=episodes, optimizer=optimizer)

[Qibo 0.1.10.dev0|INFO|2023-02-08 16:37:40]: Using tensorflow backend on /device:CPU:0


episode: 10, reward 2.460875


/home/simonebordoni/QIBO/rl-noisemodel/src/rlnoise/utils.py:15: RuntimeWarning: divide by zero encountered in double_scalars
  return 0.5*((m1-m2)**2+(v1+v2))*(1/v1+1/v2)-2


episode: 20, reward 1.921257
episode: 30, reward 1.893790
episode: 40, reward 0.985231
episode: 50, reward 2.876722
episode: 60, reward 2.677763
episode: 70, reward 2.463899
episode: 80, reward 1.846459
episode: 90, reward 1.969079
episode: 100, reward 1.961109
INFO:tensorflow:Assets written to: /home/simonebordoni/QIBO/rl-noisemodel/models/model_1q/assets


2023-02-08 16:41:29.689691: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
